# Kaggle Competition 

In [16]:
import pandas as pd
import numpy as np

In [17]:
import pytorch_lightning as pl
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [18]:
train_df =  pd.read_parquet("data/train_data.parquet")

In [19]:
train_df.shape

(11774752, 101)

In [46]:
valid_df =  pd.read_parquet("data/validate_data.parquet")

In [47]:
valid_df.shape

(639848, 101)

In [21]:
feature_cols = [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]

#select target values
target_cols = ["responder_6"]

# select the weight values
weight_cols = ["weight"]

In [97]:
#finsh
class CustomDataset(Dataset):
    def __init__(self, dataframe, accerlerator):
        # Store each part of the dataframe that needed as a tensor
        self.features = torch.FloatTensor(dataframe[feature_cols].values).to(accerlerator)
        self.labels = torch.FloatTensor(dataframe[target_cols].values).to(accerlerator)
        self.weights = torch.FloatTensor(dataframe[weight_cols].values).to(accerlerator)


    def __len__(self):
        # Returb the length of the dataset
        return len(self.labels)

    def __getitem__(self, idx):
        # return the data at a given index
        # x = all features defined in the feature_cols
        # y = the target values that needed to be predicted definded in the target_cols
        # w = the weight values for calculating the loss defined in the weight_cols
        x = self.features[idx]
        y = self.labels[idx]
        w = self.weights[idx]
        return x, y, w

In [104]:
res  = CustomDataset(train_df[:1000], "cpu")

In [108]:
res.features.shape[0]

1000

In [98]:
# finish
# problem that the train_dataset is somehow not defined after the setup function
# the class CustomDataset is not called in the setup function
# the class CustomDataset can be called out side the function
class DataModule(pl.LightningDataModule):
    def __init__(self, dataframe,  batch_size, valid_df = None, accelerator='cpu'):
        super().__init__()
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.dates = dataframe["date_id"].unique()
        self.accelerator = accelerator
        self.train_dataset = None
        self.valid_df = None
        if valid_df is not None:
            self.valid_df = valid_df
        self.val_dataset = None
        
    def setup(self, fold=0, N_fold=5, stage=None):
        selected_dates = [date for ii, date in enumerate(self.dates) if ii % N_fold != fold] #leave one section out
        df_train = self.dataframe.loc[self.df['date_id'].isin(selected_dates)]
        self.train_dataset = CustomDataset(df_train, self.accelerator)
        if self.valid_df is not None:
            df_valid = self.valid_df
            self.val_dataset = CustomDataset(df_valid, self.accelerator)

    def train_dataloader(self, n_workers=0):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=n_workers)

    def val_dataloader(self, n_workers=0):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=n_workers)


In [61]:
# finish
class MyModel(pl.LightningModule):
    def __init__(self, input_dim, hidden_dims, dropouts, lr, weight_decay):
        super().__init__()
        self.save_hyperparameters()
        layers = []
        in_dim = input_dim
        for i, hidden_dim in enumerate(hidden_dims):
            layers.append(nn.BatchNorm1d(in_dim))
            if i > 0:
                layers.append(nn.SiLU())
            if i < len(dropouts):
                layers.append(nn.Dropout(dropouts[i]))
            layers.append(nn.Linear(in_dim, hidden_dim))
            # layers.append(nn.ReLU())
            in_dim = hidden_dim
        layers.append(nn.Linear(in_dim, 1))  # 输出层
        layers.append(nn.Tanh())
        self.model = nn.Sequential(*layers)
        self.lr = lr
        self.weight_decay = weight_decay
        self.validation_step_outputs = []
    
    def forward(self, x):
        return 5 * self.model(x).squeeze(-1)
    
    def validation_step(self, batch):
        x, y, w = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y, reduction='none') * w
        loss = loss.mean()
        self.log('val_loss', loss, on_step=False, on_epoch=True, batch_size=x.size(0))
        self.validation_step_outputs.append((y_hat, y, w))
        return loss

    def on_validation_epoch_end(self):
        """Calculate validation WRMSE at the end of the epoch."""
        y = torch.cat([x[1] for x in self.validation_step_outputs]).cpu().numpy()
        if self.trainer.sanity_checking:
            prob = torch.cat([x[0] for x in self.validation_step_outputs]).cpu().numpy()
        else:
            prob = torch.cat([x[0] for x in self.validation_step_outputs]).cpu().numpy()
            weights = torch.cat([x[2] for x in self.validation_step_outputs]).cpu().numpy()
            # r2_val
            val_r_square = r2_val(y, prob, weights)
            self.log("val_r_square", val_r_square, prog_bar=True, on_step=False, on_epoch=True)
        self.validation_step_outputs.clear()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5,
                                                               verbose=True)
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss',
            }
        }

    def on_train_epoch_end(self):
        if self.trainer.sanity_checking:
            return
        epoch = self.trainer.current_epoch
        metrics = {k: v.item() if isinstance(v, torch.Tensor) else v for k, v in self.trainer.logged_metrics.items()}
        formatted_metrics = {k: f"{v:.5f}" for k, v in metrics.items()}
        print(f"Epoch {epoch}: {formatted_metrics}")

In [99]:
data_module = DataModule(train_df, batch_size=1000,valid_df= valid_df ,accelerator= 'cpu')